### Import the required lib

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

import urllib
import cv2
import time
import urllib.request

### Add parent folder to path as parent folder contains required file to run this modal

In [ ]:
sys.path.append("..")

### add libs form parent folder

In [ ]:

from utils import label_map_util
from utils import visualization_utils as vis_util

### Initialize the variables
In this example, I have used Android IP camera to get the live feed. So I need to configure the OpenCV to feed to live video to model directly from my phone (IP:192.168.1.4 )

By default an "SSD with Mobilenet" model is used here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
if tf.__version__ < '1.4.0':
    raise ImportError(
        'Please upgrade your tensorflow installation to v1.4.* or later!')
    
    
url = 'http://192.168.1.4:8080/video'


# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90  #number of class present in the training.

outFile = "/home/guru/test.mp4"
# fourcc stands for four character code
codec = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
framerate = 30
resolution = (640, 480)

### Download Data 

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

### Load the Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Load label map
Label map is used to identify the correct value from the key produced by the convolutional network

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


###  Run the model

In [ ]:
video_file = cv2.VideoWriter(outFile, codec, framerate, resolution)

# if you are getting CUDA MEMORY OVERFLOW error( for tensorflow-gpu version), add this as config in session.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

bytes = b''
with detection_graph.as_default():
    with tf.Session(config=config, graph=detection_graph) as sess:

        stream = urllib.request.urlopen(url)

        while (True):
            bytes += stream.read(1024)
            a = bytes.find(b'\xff\xd8')
            b = bytes.find(b'\xff\xd9')
            if a != -1 and b != -1:
                jpg = bytes[a:b + 2]
                bytes = bytes[b + 2:]
                image_np = cv2.imdecode(np.fromstring(
                    jpg, dtype=np.uint8), cv2.IMREAD_COLOR)

                #input and output Tensors for graph
                image_tensor = detection_graph.get_tensor_by_name(
                    'image_tensor:0')
                
                detection_boxes = detection_graph.get_tensor_by_name(
                    'detection_boxes:0')
                
                detection_scores = detection_graph.get_tensor_by_name(
                    'detection_scores:0')
                
                detection_classes = detection_graph.get_tensor_by_name(
                    'detection_classes:0')
                
                num_detections = detection_graph.get_tensor_by_name(
                    'num_detections:0')

                # Expand array to shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                
                # Start the detection
                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores,
                        detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                # Disply the detected object
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)

                video_file.write(image_np)
                cv2.imshow('live_detection', image_np)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
                    cv2.destroyAllWindows()
                    cap.release()
